In [ ]:
import base64
import requests


def run(fname, app_id, app_code):

    inputdata = open(fname,"rb").read()
    #print("inputdata.read:", inputdata)
    inputdata = base64.b64encode(inputdata)
    #print("b64encoded inputdata:", inputdata)

    url = "https://rme.api.here.com/2/matchroute.json"
    params = {
        "app_id": app_id,
        "app_code": app_code,
        "routemode": "car",
        "file": inputdata
    }

    result = requests.get(url=url, params=params)

    print("status code:", result.status_code)

    content = result.json()
    #print("content:", content)

    routes, traces, linkmap = extract_matched_info(content)

    print("routes:", routes)

    print("traces:")
    for t in traces: print(t)

    print("linkmap:")
    for k, v in linkmap.items(): print(k, v)

    return


def extract_matched_info(content):

    linkmap = {}
    routes = []
    for link in content["RouteLinks"]:
        linkId = link["linkId"]
        routes.append(linkId)
        length = float(link["linkLength"])
        fc = int(link["functionalClass"])
        shape = []
        shapes = link["shape"].strip().split(" ")
        nshapes = len(shapes)
        for i in range(0, nshapes, 2):
            shape.append((float(shapes[i]), float(shapes[i+1])))
        linkmap[linkId] = [fc, length, shape]

    traces = []
    for trace in content["TracePoints"]:
        lat = float(trace["lat"])
        lon = float(trace["lon"])
        linkIdMatched = trace["linkIdMatched"]
        latMatched = float(trace["latMatched"])
        lonMatched = float(trace["lonMatched"])
        matchOffsetOnLink = float(trace["matchOffsetOnLink"])
        matchDistance = float(trace["matchDistance"])
        traces.append((lat, lon, linkIdMatched, latMatched, lonMatched, matchOffsetOnLink, matchDistance))

    return routes, traces, linkmap

In [4]:
fname = "AndrosensorDataYaejoon.gpx"

# obtain app_id, app_code from developer.here.com
app_id = "gtswh2sa822DHthznV1X"
app_code = "zd2FnIRxq25-BNHJSqgjtg"

run(fname, app_id, app_code)

status code: 414


JSONDecodeError: Expecting value: line 1 column 1 (char 0)